In [1]:
import pandas as pd
import numpy as np

from PIL import Image 
import os
import cv2

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [2]:
# Storing the images path
image_path = 'dataset/color600'

In [3]:
# convert the images and save them in a array

# without regularization

X = [] # stores the image 
y = [] # stores the plants class
diseases = [] #stores the diseases of the plants

for folder_name, _,filenames in os.walk(image_path):
    if folder_name !="dataset/color" and '.DS_Store' not in filenames:
        for file in filenames:
            if file != 'desktop.ini':
                file_path = folder_name +"/"+ file
                image = Image.open(file_path)
                image = np.array(image.convert('RGB')) # converts images to RGB as an array
                #image = np.array(image.resize((28,28))) #resizes images to (28,28)
                X.append(image)
                y.append(os.path.basename(folder_name.split('\\')[1].split('___')[0]))
                diseases.append(os.path.basename(folder_name.split('\\')[1].split('___')[1]))
            else:
                pass
    else:
        pass
    
# changing the target variable into a binary category (healthy, not heathy)
y = [0 if x == 'healthy' else 1 for x in diseases]
X = np.array(X)
y = np.array(y)

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [4]:
#Model_1: Model for binary classification without regularization

from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

model_1 = Sequential()

model_1.add(Conv2D(filters = 16,
                     input_shape = (256,256,3),
                     kernel_size = (3,3),
                     activation = 'relu'))

model_1.add(MaxPooling2D(pool_size = (2, 2)))

model_1.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))

model_1.add(MaxPooling2D(pool_size = (2, 2)))

model_1.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_1.add(MaxPooling2D(pool_size = (2, 2)))

model_1.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_1.add(MaxPooling2D(pool_size = (2, 2)))

model_1.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_1.add(MaxPooling2D(pool_size = (2, 2)))

model_1.add(Dropout(rate = 0.2))

model_1.add(Flatten())

model_1.add(Dense(units = 512,
                    activation = 'relu'))

model_1.add(Dense(units = 1,
                     activation='sigmoid'))

model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_1_history = model_1.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
248/248 [==============================] - 148s 592ms/step - loss: 1.3845 - accuracy: 0.7305 - val_loss: 0.4693 - val_accuracy: 0.7295
Epoch 2/50
248/248 [==============================] - 141s 569ms/step - loss: 0.3147 - accuracy: 0.8492 - val_loss: 0.3015 - val_accuracy: 0.8424
Epoch 3/50
248/248 [==============================] - 142s 574ms/step - loss: 0.2076 - accuracy: 0.9066 - val_loss: 0.1624 - val_accuracy: 0.9359
Epoch 4/50
248/248 [==============================] - 149s 601ms/step - loss: 0.1579 - accuracy: 0.9326 - val_loss: 0.1373 - val_accuracy: 0.9447
Epoch 5/50
248/248 [==============================] - 147s 592ms/step - loss: 0.1472 - accuracy: 0.9391 - val_loss: 0.1243 - val_accuracy: 0.9513
Epoch 6/50
248/248 [==============================] - 146s 590ms/step - loss: 0.1153 - accuracy: 0.9530 - val_loss: 0.1291 - val_accuracy: 0.9515
Epoch 7/50
248/248 [==============================] - 147s 594ms/step - loss: 0.1238 - accuracy: 0.9522 - val_loss: 0.1144 -

In [6]:
# No regularization 
# predicting the the plants of the test set
y_pred_model_1 = model_1.predict(X_test)

# transforming the predictions into 1 and 0 by splitting at 0.5
y_pred = np.array([0 if x < 0.5 else 1 for x in y_pred_model_1])

# classification report
print(classification_report(y_test, y_pred))

# confusion matrix 
pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predicted'])

124/124 [==============================] - 10s 76ms/step
              precision    recall  f1-score   support

           0       0.96      0.95      0.95      1095
           1       0.98      0.98      0.98      2865

    accuracy                           0.97      3960
   macro avg       0.97      0.97      0.97      3960
weighted avg       0.97      0.97      0.97      3960



Predicted,0,1
Real,,
0,1041,54
1,48,2817


In [7]:
# convert the images and save them in a array

# MinMax norm normalization

X = [] # stores the image 
#y = [] # stores the plants class
diseases = [] #stores the diseases of the plants

for folder_name, _,filenames in os.walk(image_path):
    if folder_name !="dataset/color600" and '.DS_Store' not in filenames:
        for file in filenames:
            if file != 'desktop.ini':
                file_path = folder_name +"/"+ file
                image = Image.open(file_path)
                image = np.array(image.convert('RGB')) # converts images to RGB and to an numpy array
                image= cv2.normalize(image, None, 0, 1.0,
                                    cv2.NORM_MINMAX, dtype=cv2.CV_32F) # regularization
                X.append(image)
                #y.append(os.path.basename(folder_name.split('\\')[1].split('___')[0]))
                diseases.append(os.path.basename(folder_name.split('\\')[1].split('___')[1]))
            else:
                pass
    else:
        pass
    
# changing the target variable into a binary category (healthy, not healthy)
y = [0 if x == 'healthy' else 1 for x in diseases]
X = np.array(X)
y = np.array(y)

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
#Model_2:  Model for binary classification with MinMax norm regularization

from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

model_2 = Sequential()

model_2.add(Conv2D(filters = 16,
                     input_shape = (256,256,3),
                     kernel_size = (3,3),
                     activation = 'relu'))

model_2.add(MaxPooling2D(pool_size = (2, 2)))

model_2.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))

model_2.add(MaxPooling2D(pool_size = (2, 2)))

model_2.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_2.add(MaxPooling2D(pool_size = (2, 2)))

model_2.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_2.add(MaxPooling2D(pool_size = (2, 2)))

model_2.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_2.add(MaxPooling2D(pool_size = (2, 2)))

model_2.add(Dropout(rate = 0.2))

model_2.add(Flatten())

model_2.add(Dense(units = 512,
                    activation = 'relu'))

model_2.add(Dense(units = 1,
                     activation='sigmoid'))

model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_2_history = model_2.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
248/248 [==============================] - 179s 708ms/step - loss: 0.3795 - accuracy: 0.8109 - val_loss: 0.1693 - val_accuracy: 0.9301
Epoch 2/50
248/248 [==============================] - 170s 685ms/step - loss: 0.1685 - accuracy: 0.9297 - val_loss: 0.1335 - val_accuracy: 0.9432
Epoch 3/50
248/248 [==============================] - 172s 694ms/step - loss: 0.1247 - accuracy: 0.9496 - val_loss: 0.0906 - val_accuracy: 0.9646
Epoch 4/50
248/248 [==============================] - 170s 686ms/step - loss: 0.0870 - accuracy: 0.9658 - val_loss: 0.0651 - val_accuracy: 0.9747
Epoch 5/50
248/248 [==============================] - 171s 692ms/step - loss: 0.0672 - accuracy: 0.9747 - val_loss: 0.0767 - val_accuracy: 0.9699
Epoch 6/50
248/248 [==============================] - 172s 695ms/step - loss: 0.0586 - accuracy: 0.9780 - val_loss: 0.0542 - val_accuracy: 0.9811
Epoch 7/50
248/248 [==============================] - 171s 690ms/step - loss: 0.0387 - accuracy: 0.9852 - val_loss: 0.0477 -

In [9]:
# MinMax norm regularization
# predicting the the plants of the test set
y_pred_model_2 = model_2.predict(X_test)

# transforming the predictions into 1 and 0 by splitting at 0.5
y_pred = np.array([0 if x < 0.5 else 1 for x in y_pred_model_2])

# classification report 
print(classification_report(y_test, y_pred))

# confusion matrix 
pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predicted'])

124/124 [==============================] - 12s 91ms/step
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1095
           1       1.00      0.99      0.99      2865

    accuracy                           0.99      3960
   macro avg       0.99      0.99      0.99      3960
weighted avg       0.99      0.99      0.99      3960



Predicted,0,1
Real,,
0,1085,10
1,25,2840


In [3]:
# convert the images and save them in a array

# MinMax norm binary normalization

X = [] # stores the image 
#y = [] # stores the plants class
diseases = [] #stores the diseases of the plants

for folder_name, _,filenames in os.walk(image_path):
    if folder_name !="dataset/color" and '.DS_Store' not in filenames:
        for file in filenames:
            if file != 'desktop.ini':
                file_path = folder_name +"/"+ file
                image = Image.open(file_path)
                image = np.array(image.convert('RGB')) # converts images to RGB as an array
                ret, thresh = cv2.threshold(image,140,255,cv2.THRESH_BINARY)
                image = cv2.normalize(thresh, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
                X.append(image)
                #y.append(os.path.basename(folder_name.split('\\')[1].split('___')[0]))
                diseases.append(os.path.basename(folder_name.split('\\')[1].split('___')[1]))
            else:
                pass
    else:
        pass
    
y = [0 if x == 'healthy' else 1 for x in diseases]
X = np.array(X)
y = np.array(y)

# split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
#Model_3:  Model for binary classification with MinMax norm binary regularization
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf

model_3 = Sequential()

model_3.add(Conv2D(filters = 16,
                     input_shape = (256,256,3),
                     kernel_size = (3,3),
                     activation = 'relu'))

model_3.add(MaxPooling2D(pool_size = (2, 2)))

model_3.add(Conv2D(filters=32, kernel_size=(3,3), activation='relu'))

model_3.add(MaxPooling2D(pool_size = (2, 2)))

model_3.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_3.add(MaxPooling2D(pool_size = (2, 2)))

model_3.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_3.add(MaxPooling2D(pool_size = (2, 2)))

model_3.add(Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

model_3.add(MaxPooling2D(pool_size = (2, 2)))

model_3.add(Dropout(rate = 0.2))

model_3.add(Flatten())

model_3.add(Dense(units = 512,
                    activation = 'relu'))

model_3.add(Dense(units = 1,
                     activation='sigmoid'))

model_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model_3_history = model_3.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
248/248 [==============================] - 164s 657ms/step - loss: 0.3274 - accuracy: 0.8427 - val_loss: 0.2649 - val_accuracy: 0.8848
Epoch 2/50
248/248 [==============================] - 166s 669ms/step - loss: 0.1684 - accuracy: 0.9288 - val_loss: 0.1389 - val_accuracy: 0.9437
Epoch 3/50
248/248 [==============================] - 165s 665ms/step - loss: 0.1322 - accuracy: 0.9458 - val_loss: 0.1324 - val_accuracy: 0.9455
Epoch 4/50
248/248 [==============================] - 166s 672ms/step - loss: 0.1096 - accuracy: 0.9557 - val_loss: 0.1182 - val_accuracy: 0.9487
Epoch 5/50
248/248 [==============================] - 166s 670ms/step - loss: 0.0840 - accuracy: 0.9655 - val_loss: 0.1085 - val_accuracy: 0.9571
Epoch 6/50
248/248 [==============================] - 167s 675ms/step - loss: 0.0764 - accuracy: 0.9690 - val_loss: 0.1166 - val_accuracy: 0.9551
Epoch 7/50
248/248 [==============================] - 167s 670ms/step - loss: 0.0548 - accuracy: 0.9769 - val_loss: 0.1109 -

In [7]:
#MinMax norm binary regularization
# predicting the the plants of the test set
y_pred_model_3 = model_3.predict(X_test)

# transforming the predictions into 1 and 0 by splitting at 0.5
y_pred = np.array([0 if x < 0.5 else 1 for x in y_pred_model_3])

# classification report 
print(classification_report(y_test, y_pred))

# confusion matrix 
pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predicted'])

124/124 [==============================] - 13s 95ms/step
              precision    recall  f1-score   support

           0       0.94      0.95      0.94      1095
           1       0.98      0.98      0.98      2865

    accuracy                           0.97      3960
   macro avg       0.96      0.96      0.96      3960
weighted avg       0.97      0.97      0.97      3960



Predicted,0,1
Real,,
0,1037,58
1,70,2795
